In [3]:
import requests
from requests.auth import HTTPBasicAuth
import json
import xml.etree.ElementTree as ET
from pandas import json_normalize
import sandkey

In [12]:
username = 'sapozhnikov.i@northeastern.edu'
password = 'Shark3Gum'
base_url = 'https://webservices.iso-ne.com/api/v1.1'

In [13]:
# edit this
date = '20241012'
application = f'genfuelmix/day/{date}'
endpoint = f'/{application}.json'

url = base_url + endpoint
print(url)

https://webservices.iso-ne.com/api/v1.1/genfuelmix/day/20241012.json


In [14]:
# Make the request
response = requests.get(url, auth=HTTPBasicAuth(username, password))

# Check if the request was successful
if response.status_code == 200:
    if endpoint.endswith('.json'):
        # Handle JSON response
        data = response.json()
        print(json.dumps(data, indent=2))
    elif endpoint.endswith('.xml'):
        # Handle XML response
        root = ET.fromstring(response.content)
        print(ET.tostring(root, encoding='utf8').decode('utf8'))
else:
    print('Failed to retrieve data:', response.status_code)

{
  "GenFuelMixes": {
    "GenFuelMix": [
      {
        "BeginDate": "2024-10-12T00:05:37.000-04:00",
        "GenMw": 241,
        "FuelCategoryRollup": "Hydro",
        "FuelCategory": "Hydro",
        "MarginalFlag": "N"
      },
      {
        "BeginDate": "2024-10-12T00:05:37.000-04:00",
        "GenMw": 6261,
        "FuelCategoryRollup": "Natural Gas",
        "FuelCategory": "Natural Gas",
        "MarginalFlag": "Y"
      },
      {
        "BeginDate": "2024-10-12T00:05:37.000-04:00",
        "GenMw": 1229,
        "FuelCategoryRollup": "Nuclear",
        "FuelCategory": "Nuclear",
        "MarginalFlag": "N"
      },
      {
        "BeginDate": "2024-10-12T00:05:37.000-04:00",
        "GenMw": 71,
        "FuelCategoryRollup": "Other",
        "FuelCategory": "Other",
        "MarginalFlag": "N"
      },
      {
        "BeginDate": "2024-10-12T00:05:37.000-04:00",
        "GenMw": 28,
        "FuelCategoryRollup": "Renewables",
        "FuelCategory": "Landfill Gas",
  

In [10]:
df = json_normalize(data['GenFuelMixes']['GenFuelMix'])

In [11]:
df.head()

,BeginDate,GenMw,FuelCategoryRollup,FuelCategory,MarginalFlag
0,2024-10-12T00:05:37.000-04:00,241,Hydro,Hydro,N
1,2024-10-12T00:05:37.000-04:00,6261,Natural Gas,Natural Gas,Y
2,2024-10-12T00:05:37.000-04:00,1229,Nuclear,Nuclear,N
3,2024-10-12T00:05:37.000-04:00,71,Other,Other,N
4,2024-10-12T00:05:37.000-04:00,28,Renewables,Landfill Gas,N


In [28]:
col2 = 'FuelCategoryRollup'
col1 = 'FuelCategory'
word = 'NonRenewable'

df[col2] = df.apply(lambda row: word if row[col1] == row[col2] else row[col2], axis=1)

In [29]:
sandkey.make_sankey(df, columns=['FuelCategory', 'FuelCategoryRollup'], vals_col=['GenMw'])